In [1]:
import pandas as pd
from statistics import *
from math import isnan
import matplotlib.pyplot as plt # библиотека визуализации
from scipy import stats # библиотека для расчетов
from sklearn.metrics import matthews_corrcoef 
import seaborn as sns
import plotly
import plotly.express as px

В футболе статистические функции могут представлять собой хорошо известные модели, такие как модель регрессии Пуассона или модель Диксона и Коулза.

In [2]:
season = {
    "bundes_22-23":[ 1736824, 1737122,298 ],
    "Prem_Leag_22-23":[1736219,1736589,370],
    "Premiere_22-23":[1744303,1744668,365],
    "Series_A_22-23":[1747394,1747768,374]
}

In [3]:
df = pd.read_csv("data/bundes_22-23.csv")

In [4]:
display(df.head())
df.info()

,Unnamed: 0,id,общая информация,Дата,Команда1,счет1,Команда2,счет2,Температура,Признак погоды,...,Офсайды (соперник) 1,Офсайды (соперник) 2,Желтые карточки 1,Желтые карточки 2,Желтые карточки (соперник) 1,Желтые карточки (соперник) 2,Красные карточки 1,Красные карточки 2,Красные карточки (соперник) 1,Красные карточки (соперник) 2
0,0,1736825,"Бундеслига, 1-й тур, регулярный сезон, 05.08.2...",05.08.2022 21:35,Айнтрахт Франкфурт,1,Бавария,6,+29°C,солнечно,...,2.00,2.5,0.88,1.90,1.00,1.50,0.0,0.10,0.25,0.0
1,1,1736826,"Бундеслига, 1-й тур, регулярный сезон, 06.08.2...",06.08.2022 19:30,Боруссия Дортмунд,1,Байер,0,+27°C,солнечно,...,1.63,1.0,0.75,0.89,1.25,1.00,0.0,0.00,0.00,0.0
2,2,1736827,"Бундеслига, 1-й тур, регулярный сезон, 06.08.2...",06.08.2022 16:30,Унион Берлин,3,Герта,1,+28°C,солнечно,...,0.67,2.0,1.20,1.75,0.80,0.75,0.0,0.25,0.00,0.0
3,3,1736828,"Бундеслига, 1-й тур, регулярный сезон, 07.08.2...",07.08.2022 18:30,Кёльн,3,Шальке,1,+30°C,солнечно,...,1.40,2.5,2.00,0.40,1.67,0.00,0.0,0.00,0.00,0.0
4,4,1736829,"Бундеслига, 1-й тур, регулярный сезон, 06.08.2...",06.08.2022 16:30,Боруссия М,3,Хоффенхайм,1,+27°C,солнечно,...,2.33,2.0,0.29,2.00,0.57,1.40,0.0,0.00,0.00,0.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 298 entries, 0 to 297
Data columns (total 73 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Unnamed: 0                      298 non-null    int64  
 1   id                              298 non-null    int64  
 2   общая информация                298 non-null    object 
 3   Дата                            298 non-null    object 
 4   Команда1                        298 non-null    object 
 5   счет1                           298 non-null    object 
 6   Команда2                        298 non-null    object 
 7   счет2                           298 non-null    object 
 8   Температура                     298 non-null    object 
 9   Признак погоды                  298 non-null    object 
 10  поб1                            298 non-null    float64
 11  X                               298 non-null    float64
 12  поб2                            298 

In [5]:
fig = px.imshow(
    df.corr(),
    text_auto=True,
    height=1400,
    width=1400
)
fig.show()

In [6]:
df[["Температура","Признак погоды"]].value_counts()

Температура  Признак погоды                                          
+27°C        солнечно                                                    71
+28°C        солнечно                                                    65
+29°C        солнечно                                                    51
+28°C        переменная облачность                                       34
+30°C        солнечно                                                    31
+29°C        умеренный или сильный ливневый дождь                        17
+30°C        в отдельных районах умеренный или сильный дождь с грозой    15
+24°C        солнечно                                                    14
dtype: int64

In [7]:
df['Признак погоды'].value_counts()

солнечно                                                    232
переменная облачность                                        34
умеренный или сильный ливневый дождь                         17
в отдельных районах умеренный или сильный дождь с грозой     15
Name: Признак погоды, dtype: int64

In [8]:
df[df["счет1"] == '-']["id"]

136    1736961
288    1737113
Name: id, dtype: int64

In [9]:
display(df[df['id'] == 1736961])
display(df[df['id'] == 1737113])

,Unnamed: 0,id,общая информация,Дата,Команда1,счет1,Команда2,счет2,Температура,Признак погоды,...,Офсайды (соперник) 1,Офсайды (соперник) 2,Желтые карточки 1,Желтые карточки 2,Желтые карточки (соперник) 1,Желтые карточки (соперник) 2,Красные карточки 1,Красные карточки 2,Красные карточки (соперник) 1,Красные карточки (соперник) 2
136,136,1736961,"Суперкубок, финал, 12.08.2023 21:45",12.08.2023 21:45,Бавария,-,РБ Лейпциг,-,+28°C,солнечно,...,0.8,1.2,1.5,2.4,1.9,2.6,0.0,0.2,0.0,0.0


,Unnamed: 0,id,общая информация,Дата,Команда1,счет1,Команда2,счет2,Температура,Признак погоды,...,Офсайды (соперник) 1,Офсайды (соперник) 2,Желтые карточки 1,Желтые карточки 2,Желтые карточки (соперник) 1,Желтые карточки (соперник) 2,Красные карточки 1,Красные карточки 2,Красные карточки (соперник) 1,Красные карточки (соперник) 2
288,288,1737113,"Суперкубок, финал, 12.08.2023 21:45",12.08.2023 21:45,Бавария,-,РБ Лейпциг,-,+28°C,солнечно,...,0.8,1.2,1.5,2.4,1.9,2.6,0.0,0.2,0.0,0.0


In [ ]:
corr_with_target = df.corr()['MEDV'].abs().sort_values()
corr_with_target